In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

# Ввод датасетов

In [2]:
df_2018 = pd.read_csv('archive/kaggle-survey-2018/multipleChoiceResponses.csv',
                      encoding='latin1', low_memory=False)

In [3]:
df_2019 = pd.read_csv('archive/kaggle-survey-2019/multiple_choice_responses.csv',
                      encoding='latin1', low_memory=False)

In [4]:
df_2020 = pd.read_csv('archive/kaggle-survey-2020/kaggle_survey_2020_responses.csv',
                      encoding='latin1', low_memory=False)

In [5]:
df_2021 = pd.read_csv('archive/kaggle-survey-2021/kaggle_survey_2021_responses.csv',
                      encoding='latin1', low_memory=False)

In [6]:
df_2022 = pd.read_csv('archive/kaggle-survey-2022/kaggle_survey_2022_responses.csv',
                      encoding='latin1', low_memory=False)

In [7]:
df_2025 = pd.read_csv('archive/kaggle-survey-2025/multipleChoiceResponses.csv',
                      encoding='latin1', low_memory=False)

# Column mapping

In [8]:
datasets = {
        '2018': df_2018, 
        '2019': df_2019,
        '2020': df_2020,
        '2021': df_2021,
        '2022': df_2022,
        '2025': df_2025
    }

In [9]:
def normalize_df(df):
    name2questions = df.iloc[0].to_list()
    df.columns = name2questions
    df.drop(0, inplace=True)

    return df

In [10]:
for key, df in datasets.items():
    df = normalize_df(df)

In [11]:
def auto_create_mapping(columns_list):
    mapping = {}
    
    for col in columns_list:
        simplified = simplify_column_name(col)
        mapping[col] = simplified
    
    return mapping

def simplify_column_name(column_name):
    name = column_name.lower()
    
    patterns_to_remove = [
        r'which of the following',
        r'select all that apply',
        r'selected choice',
        r'other - text',
        r'approximate \$usd',
        r'include text response',
        r'in the last 5 years',
        r'at work or school',
        r'what is your',
        r'how many',
        r'does your',
        r'select any',
        r'approximate what percent',
        r'how long',
        r'for how many',
        r'do you consider',
        r'what specific',
        r'what programming',
        r'what machine learning',
        r'what data visualization',
        r'of the choices that you selected in the previous question',
        r'which specific',
        r'what is the type',
        r'where do you',
        r'during a typical',
        r'what percentage',
        r'on which online platforms',
        r'who/what are your',
        r'how do you perceive',
        r'which better demonstrates',
        r'what metrics',
        r'what do you find most difficult',
        r'in what circumstances',
        r'what methods',
        r'what tools and methods',
        r'what barriers'
    ]
    
    for pattern in patterns_to_remove:
        name = re.sub(pattern, '', name)
    
    name = re.sub(r'[^a-z0-9\s]', ' ', name)
    name = re.sub(r'\s+', '_', name.strip())
    
    if len(name) > 50:
        keywords = ['gender', 'age', 'country', 'education', 'major', 'job', 'title', 
                   'industry', 'experience', 'compensation', 'tool', 'language', 'framework',
                   'ide', 'notebook', 'cloud', 'database', 'ml', 'ai', 'data', 'viz',
                   'time', 'year', 'skill', 'challenge', 'method', 'platform']
        
        for keyword in keywords:
            if keyword in name:
                name = keyword + '_' + name.split('_')[-1]
                break
    
    return name


def auto_mapper(df):
    auto_mapping = auto_create_mapping(df.columns)
    return df.rename(columns=auto_mapping)

In [12]:
mapped_datasets = {}
for key, df in datasets.items():
    df['survey_year'] = key
    mapped_datasets[key] = auto_mapper(df)

# Объединение дубликатов

In [13]:
import tqdm

In [14]:
def efficient_merge_duplicates(df):
    unique_columns = df.columns.unique()
    merged_data = {}
    
    for col in tqdm.tqdm(unique_columns):
        duplicate_cols = [c for c in df.columns if c == col]
        
        if len(duplicate_cols) == 1:
            merged_data[col] = df[col]
        else:
            duplicate_df = df[duplicate_cols]
            
            merged_col = duplicate_df.apply(
                lambda x: x.dropna().iloc[0] if x.dropna().any() else np.nan, 
                axis=1
            )
            merged_data[col] = merged_col
    return pd.DataFrame(merged_data, index=df.index)

In [15]:
normilized_datasets = {}
for key, df in mapped_datasets.items():
    normilized_datasets[key] = efficient_merge_duplicates(df)

100%|█████████████████████████████████████████| 339/339 [00:24<00:00, 13.65it/s]


# Создание датасета

In [16]:
years = list(normilized_datasets.keys())
common_cols = list(set.intersection(*(set(normilized_datasets[year].columns) for year in years)))

for year in years:
    normilized_datasets[year] = normilized_datasets[year][common_cols]
    
combined_df = pd.concat(normilized_datasets.values(), ignore_index=True)
combined_df.to_csv('kaggle_survey_2018-2025.csv', index=False)